# Chapter 4: Analyzing tabular data with pyspark.sql

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/09 21:57:33 WARN Utils: Your hostname, LAPTOP-CDHH1LA0 resolves to a loopback address: 127.0.1.1; using 172.23.108.60 instead (on interface eth0)
24/06/09 21:57:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 21:57:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# generate dataframe from list of lists
my_grocery_list = [
    ["Bannana", 2, 1.74],
    ["Apple", 4, 2.04],
    ["Carrot", 1, 1.09],
    ["Cake", 1, 10.99]
]

df_grocery_list = spark.createDataFrame(my_grocery_list, ["Item", "Quantity", "Price"])

df_grocery_list.printSchema()

root
 |-- Item: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: double (nullable = true)



In [4]:
df_grocery_list.select(F.lower(F.col("Item")).alias("litem"), F.col("Price")).show()

+-------+-----+
|  litem|Price|
+-------+-----+
|bannana| 1.74|
|  apple| 2.04|
| carrot| 1.09|
|   cake|10.99|
+-------+-----+



In [5]:
import os 
DIRECTORY = "./data/broadcast_logs"
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8_sample.CSV"), # the path to the target file is the only mandatory param
    sep="|", # field delimiter "," by default but this can have issues if dealing with text that contains commas
    header = True, # indicates if the file has a header row, can manually set column names with "schema" param
    inferSchema=True, # tells python to guess at schema, can manually specify schema as well
    timestampFormat = "yyyy-MM-dd", # tells python how to parse timestamps
)

In [6]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [7]:
logs.show(3)

24/06/09 21:57:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+--------------+------------+----------+----------+-------------------

In [8]:
sample = spark.read.csv(os.path.join("data", "sample.csv"), sep=",", header=True, quote="$", inferSchema=True)
sample.printSchema()
sample.show()

root
 |-- Item: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)

+---------------+--------+-----+
|           Item|Quantity|Price|
+---------------+--------+-----+
|Banana, organic|       1| 0.99|
|           Pear|       7| 1.24|
|Cake, chocolate|       1| 14.5|
+---------------+--------+-----+



In [9]:
print(logs.columns)

['BroadcastLogID', 'LogServiceID', 'LogDate', 'SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID', 'CategoryID', 'ClosedCaptionID', 'CountryOfOriginID', 'DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID', 'ProgramClassID', 'FilmClassificationID', 'ExhibitionID', 'Duration', 'EndTime', 'LogEntryDate', 'ProductionNO', 'ProgramTitle', 'StartTime', 'Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID', 'BroadcastOriginPointID', 'CompositionID', 'Producer1', 'Producer2', 'Language1', 'Language2']


In [10]:
logs.select("BroadcastLogId", "LogServiceID", "LogDate").show(5, False) # 5 for five rows, False to not truncate

+--------------+------------+----------+
|BroadcastLogId|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



In [14]:
# you can also use "destructuring" to expand a list structure into a list of inputs
# (destructuring equivalent of the above)
logs.select(*["BroadcastLogId", "LogServiceID", "LogDate"]).show(5)

+--------------+------------+----------+
|BroadcastLogId|LogServiceID|   LogDate|
+--------------+------------+----------+
|    1196192316|        3157|2018-08-01|
|    1196192317|        3157|2018-08-01|
|    1196192318|        3157|2018-08-01|
|    1196192319|        3157|2018-08-01|
|    1196192320|        3157|2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



In [20]:
# columns of logs dataframe in groups of 3
import numpy as np

column_split = np.array_split( # split numpy array into n components
    np.array(logs.columns), # the numpy array to split
    len(logs.columns) // 3 # n (the number of subgroups) is determined via integer division by 3 (the target sublist size)
    )
print(column_split)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]


In [18]:
for x in column_split:
    logs.select(*x).show(5, False)

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|LogDate   |
+--------------+------------+----------+
|1196192316    |3157        |2018-08-01|
|1196192317    |3157        |2018-08-01|
|1196192318    |3157        |2018-08-01|
|1196192319    |3157        |2018-08-01|
|1196192320    |3157        |2018-08-01|
+--------------+------------+----------+
only showing top 5 rows

+----------+-------------------+----------------------+
|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|
+----------+-------------------+----------------------+
|1         |4                  |NULL                  |
|2         |NULL               |NULL                  |
|3         |NULL               |NULL                  |
|4         |NULL               |NULL                  |
|5         |NULL               |NULL                  |
+----------+-------------------+----------------------+
only showing top 5 rows

+----------+---------------+-----------------+
|CategoryID|ClosedCaptionID|Co

In [24]:
# in addition to selecting only the columns that you want, you can also drop columns from an existing dataframe
# this can be done with the "drop" method
print(logs.columns)
print(f"BroadcastLogID present prior to drop: {'BroadcastLogID' in logs.columns}")

logs = logs.drop("BroadcastLogID", "SequenceNO") # logs.drop returns a new dataframe, so we reassign it to logs to overwrite original

print(f"BroadcastLogID present post drop: {'BroadcastLogID' in logs.columns}")


['LogServiceID', 'LogDate', 'AudienceTargetAgeID', 'AudienceTargetEthnicID', 'CategoryID', 'ClosedCaptionID', 'CountryOfOriginID', 'DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID', 'ProgramClassID', 'FilmClassificationID', 'ExhibitionID', 'Duration', 'EndTime', 'LogEntryDate', 'ProductionNO', 'ProgramTitle', 'StartTime', 'Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID', 'BroadcastOriginPointID', 'CompositionID', 'Producer1', 'Producer2', 'Language1', 'Language2']
BroadcastLogID present prior to drop: False
BroadcastLogID present post drop: False


In [25]:
# equivalent to the drop command but with select
logs = logs.select(*[x for x in logs.columns if x not in ["BroadcastLogID", "SequenceNO"]])

Exercise 4.2

What is the printed result of this code?

sample_frame.columns # => ['item', 'price', 'quantity', 'UPC']
 
print(sample_frame.drop('item', 'UPC', 'prices').columns)
a) ['item' 'UPC']

b) ['item', 'upc']

c) ['price', 'quantity']

d) ['price', 'quantity', 'UPC']

e) Raises an error
--------
returns ['price', 'quantity'] which is c

You can add new columns with withColumn()

In [27]:
#adding a new col with select (the long way vs withColumn)
logs.select(F.col("Duration")).show(5)

# the dtypes attribute contains the column and its type wrapped in a tuple
print(logs.select(F.col("Duration")).dtypes)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows

[('Duration', 'string')]


In [28]:
logs.dtypes

[('LogServiceID', 'int'),
 ('LogDate', 'date'),
 ('AudienceTargetAgeID', 'int'),
 ('AudienceTargetEthnicID', 'int'),
 ('CategoryID', 'int'),
 ('ClosedCaptionID', 'int'),
 ('CountryOfOriginID', 'int'),
 ('DubDramaCreditID', 'int'),
 ('EthnicProgramID', 'int'),
 ('ProductionSourceID', 'int'),
 ('ProgramClassID', 'int'),
 ('FilmClassificationID', 'int'),
 ('ExhibitionID', 'int'),
 ('Duration', 'string'),
 ('EndTime', 'string'),
 ('LogEntryDate', 'date'),
 ('ProductionNO', 'string'),
 ('ProgramTitle', 'string'),
 ('StartTime', 'string'),
 ('Subtitle', 'string'),
 ('NetworkAffiliationID', 'int'),
 ('SpecialAttentionID', 'int'),
 ('BroadcastOriginPointID', 'int'),
 ('CompositionID', 'int'),
 ('Producer1', 'string'),
 ('Producer2', 'string'),
 ('Language1', 'int'),
 ('Language2', 'int')]

In [29]:
logs.select(
    F.col("Duration"), # original column
    F.col("Duration").substr(1,2).cast("int").alias("dur_hours"), # hours # python is 1 indexed, substr takes start, length, cast to int, and alias for readability
    F.col("Duration").substr(4,2).cast("int").alias("dur_minutes"), # minutes
    F.col("Duration").substr(7,2).cast("int").alias("dur_seconds") # seconds
).distinct().show(5) # distinct used to remove duplicates to reduce clutter during display

# alternatively we could use the datetime and timedelta python constructs through udfs, but they can be slower

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:04:52.0000000|        0|          4|         52|
|00:10:06.0000000|        0|         10|          6|
|00:09:52.0000000|        0|          9|         52|
|00:04:26.0000000|        0|          4|         26|
|00:14:59.0000000|        0|         14|         59|
+----------------+---------+-----------+-----------+
only showing top 5 rows



In [30]:
# combine columns into a single duration (seconds)
logs.select(
    F.col("Duration"),
    (
        F.col("Duration").substr(1,2).cast("int") * 60 * 60
        + F.col("Duration").substr(4,2).cast("int") * 60
        + F.col("Duration").substr(7,2).cast("int")
    ).alias("Duration_seconds")
).distinct().show(5)

+----------------+----------------+
|        Duration|Duration_seconds|
+----------------+----------------+
|01:59:30.0000000|            7170|
|00:31:00.0000000|            1860|
|00:28:08.0000000|            1688|
|00:32:00.0000000|            1920|
|00:30:00.0000000|            1800|
+----------------+----------------+
only showing top 5 rows



In [31]:
#helper method to generate Duration_seconds col

def generate_Duration_seconds(raw_duration):
    return (
        F.col("Duration").substr(1,2).cast("int") * 60 * 60
        + F.col("Duration").substr(4,2).cast("int") * 60
        + F.col("Duration").substr(7,2).cast("int")
    )

In [33]:
# withColumn allows you to easily add a column to a duplicate of a dataframe based off of a combination of other columns
logs = logs.withColumn("Duration_seconds", generate_Duration_seconds(F.col("Duration")))
logs.printSchema()

# performance note, if you are making 100+ new columns, "select" will be faster than "withColumn"

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttentionID: inte

bookmark 4.4.4